## Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
import gc
from tqdm.notebook import tqdm

## Загрузим данные

In [2]:
hist_data = pd.read_csv('hist_data.csv')

## Добавим в наш хист_фрейм столбец с частотой повторений/встречаемостью item_id

In [3]:
# добавим в наш хист_фрейм столбец с частотой повторений item_id
item_freq=hist_data.item_id.value_counts().to_frame()
item_freq['id']=item_freq.index
item_freq.rename(columns={'item_id':'item_freq', 'id':'item_id'},inplace=True)
hist_data_freq = hist_data.set_index('item_id').join(item_freq.set_index('item_id'))
hist_data_freq['item_id']=hist_data_freq.index
hist_data_freq.reset_index(drop=True, inplace=True)

In [4]:
hist_data_freq.head()

,buyer_id,pav_order_id,created,count,price_sold,flag_weight_goods,weight,item_freq,item_id
0,95898494,98506705682,2021-07-01 11:55:59,1.0,14.99,False,10.31,83,202782406
1,95455775,98506807836,2021-07-01 18:40:46,5.0,14.99,False,6.96,83,202782406
2,95031957,98506929275,2021-07-02 12:17:41,1.0,14.99,False,9.41,83,202782406
3,95458981,98507248917,2021-07-03 16:48:16,2.0,14.99,False,15.75,83,202782406
4,95949542,98507339539,2021-07-04 07:28:13,4.0,12.09,False,11.13,83,202782406


## Воспользуемся предлагаемым baseline'ом

In [13]:
# Пример решения с использованием статистического подхода - подсчет 
# совстречаемостей.

import pandas as pd
import numpy as np
import gc

from collections import Counter 

hist_data = pd.read_csv('hist_data.csv')

# соберем словарь встречаемостей - какие item_id покупались чаще с 
# каждым item_id 
tmp = (
    hist_data[['item_id', 'pav_order_id']]
    .sort_values(['item_id', 'pav_order_id'])
    .merge(hist_data[['item_id', 'pav_order_id']], how='left', on=['pav_order_id'], suffixes=('', '_left'))
)
tmp = tmp[tmp['item_id'] != tmp['item_id_left']].copy()
tmp1 = tmp.groupby(['item_id'])['item_id_left'].agg(lambda x: Counter(x).most_common(10))

most_freq_dict = {k: v for (k, v) in tmp1.iteritems()}

del tmp1, tmp
gc.collect()

test = pd.read_csv('test.csv')

# из списка кандидатов по совстречаемости удаляем повторяющиеся item_id, 
# сохраняя порядок
def get_unique_recs(recs: list, top_n: int) -> list:
    rec_dict = {}
    counter = 0
    for k, v in recs:
        if k not in rec_dict:
            rec_dict[k] = v
            counter += 1
        if counter == top_n:
            break
    return list(rec_dict.keys())

def rec_by_item(item_id: int, most_freq_dict: dict) -> list:
    
    return most_freq_dict.get(item_id, None)

# для каждого item_id соберем top_n самых часто встречающихся item_id, 
# отсортируем по частоте и выберем уникальные
def rec_by_basket(basket: list, most_freq_dict: dict, top_n: int = 20) -> list:
    
    res = []
    for item in basket:
        recs = rec_by_item(item, most_freq_dict)
        if recs is not None:
            res += recs
    
    res = sorted(res, key=lambda x: x[1], reverse=True)
    
    return get_unique_recs(res, top_n)

pred = test.groupby(['pav_order_id'])['item_id'].agg([('basket', list)])
pred['preds'] = pred['basket'].map(lambda x: rec_by_basket(x, most_freq_dict=most_freq_dict))

pred['preds'].to_csv('pred.csv')

Воспользуемся полученым предсказанием/датафреймом pred.  
В этом предсказании всем товарам в корзине (из тестового набора) рекомендуются наиболее часто встречающиеся (за историю) товары "спутники", отсортированные по частоте встречаемости.  
Предположим что, для покупателей которые с нами уже работали, рекомендацию можно формировать на основании истории их покупок. Найдем/выделим в фрейме pred покупателей с которыми у нас есть история и сделаем/скорректируем для них рекомендации опираясь на те товары (отрагжируем списки) которые они уже покупали у нас.

In [29]:
# обновим pred (можно не делать)
pred = test.groupby(['pav_order_id'])['item_id'].agg([('basket', list)])
pred['preds'] = pred['basket'].map(lambda x: rec_by_basket(x, most_freq_dict=most_freq_dict))
pred.head()

,basket,preds
pav_order_id,,
4620121489,"[203164283, 204043498, 204146308, 204119602, 2...","[202820148, 202872237, 202791620, 202809628, 2..."
4620121505,"[202819114, 204074914, 202822471, 202880254, 2...","[202820148, 202872237, 202880262, 203068900, 2..."
4620121594,"[202818687, 203430473, 204016498, 203017711, 2...","[202820148, 202872237, 203059303, 202809628, 2..."
4620121684,"[203338264, 203436378, 203433668, 202812161, 2...","[202820148, 202872237, 203090014, 203090010, 2..."
4620121902,"[205768202, 202811971, 203429467, 204393593, 2...","[202820148, 203422957, 203431923, 202872237, 2..."


In [30]:
# выведем столбец pav_order_id из индексного в столбец dataframe
pred['pav_order_id']=pred.index
# сброисм индексы 
pred.reset_index(drop=True, inplace=True)
# смержим датафреймы pred и test
pred_temp=pd.merge(pred,test[['pav_order_id', 'buyer_id']], left_on='pav_order_id', right_on='pav_order_id', how='outer')
pred_temp=pred_temp[['pav_order_id','buyer_id']].drop_duplicates()
pred_new=pd.merge(pred,pred_temp[['pav_order_id', 'buyer_id']], left_on='pav_order_id', right_on='pav_order_id', how='outer')
pred_new.head()

,basket,preds,pav_order_id,buyer_id
0,"[203164283, 204043498, 204146308, 204119602, 2...","[202820148, 202872237, 202791620, 202809628, 2...",4620121489,95969605
1,"[202819114, 204074914, 202822471, 202880254, 2...","[202820148, 202872237, 202880262, 203068900, 2...",4620121505,95823859
2,"[202818687, 203430473, 204016498, 203017711, 2...","[202820148, 202872237, 203059303, 202809628, 2...",4620121594,95487550
3,"[203338264, 203436378, 203433668, 202812161, 2...","[202820148, 202872237, 203090014, 203090010, 2...",4620121684,95970193
4,"[205768202, 202811971, 203429467, 204393593, 2...","[202820148, 203422957, 203431923, 202872237, 2...",4620121902,95803805


In [31]:
# найдем топ-10 самых популярных/продаваемых товаров и будем "добивать" ими рекомендации, в случае неспособности модели сформировать именно 20 рекомендаций
top_10=hist_data['item_id'].value_counts().index.to_list()[0:20]

In [143]:
# Скорректируем pred датафрейм
# расчеты занимают 4+ аса
for i in tqdm(range(len(pred_new))):
    if pred_new.iloc[i,3] in hist_data_freq.buyer_id.value_counts().index.tolist(): 
        br=pred_new.iloc[i,3]
        t1=hist_data_freq.query('buyer_id==@br')
        t2=t1.groupby('item_id').agg(sum)
        t3=t2.sort_values(['count', 'item_freq'],ascending=False)
        t3['item_id']=t3.index
        t3.reset_index(drop=True, inplace=True)
        basket=pred_new.iloc[i,0]
        t3=t3.query("item_id not in @basket").iloc[:20, 7].tolist()
        t3.extend(top_10)
        pred_new.iat[i, pred_new.columns.get_loc('preds')] = t3[0:20]

            

  0%|          | 0/80244 [00:00<?, ?it/s]

Поправим (до требуемого формата) итоговый датафрейм, для того что бы можно было загрузить на сайт

In [32]:
pred_new.index=pred_new.pav_order_id.tolist()
pred_new.index=pred_new['pav_order_id'].to_list()
pred_new.index.rename('pav_order_id', inplace=True)
pred_new['preds'].head()

In [156]:
pred_new['preds'].to_csv('pred.csv')